Importing Libs

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score
import os
from IPython.display import display # For quick DataFrame review

# Setup for plotting
%matplotlib inline

In [54]:
import os
import json
import pandas as pd

# --- Load fixtures across all GWs ---
gw_folder = "data/gw_data"
all_fixtures = []

for file_name in os.listdir(gw_folder):
    if file_name.startswith("GW") and file_name.endswith(".json"):
        with open(os.path.join(gw_folder, file_name), "r", encoding="utf-8") as f:
            gw_data = json.load(f)
            gw_num = int(file_name.replace("GW", "").replace(".json", ""))
            for fixture in gw_data:
                fixture["gameweek"] = gw_num
            all_fixtures.extend(gw_data)

fixtures_df = pd.DataFrame(all_fixtures)

# --- Separate finished and upcoming matches ---
finished_df = fixtures_df[fixtures_df["finished"] == True].copy()
upcoming_df = fixtures_df[fixtures_df["finished"] == False].copy()

# --- Map team IDs to names (from bootstrap-static.json) ---
with open("data/bootstrap-static.json", "r", encoding="utf-8") as f:
    bootstrap_data = json.load(f)

team_id_to_name = {team["id"]: team["name"] for team in bootstrap_data["teams"]}

print(f"✅ Loaded {len(fixtures_df)} fixtures. Finished: {len(finished_df)}, Upcoming: {len(upcoming_df)}")


✅ Loaded 380 fixtures. Finished: 70, Upcoming: 310


In [55]:
from sklearn.preprocessing import LabelEncoder

# --- Encode match results ---
def match_result(row):
    if row["team_h_score"] > row["team_a_score"]:
        return "home_win"
    elif row["team_h_score"] < row["team_a_score"]:
        return "away_win"
    else:
        return "draw"

finished_df["result"] = finished_df.apply(match_result, axis=1)

# --- Features for models ---
features = ["team_h", "team_a", "team_h_difficulty", "team_a_difficulty"]

# Label encoder for result
le = LabelEncoder()
y_cls = le.fit_transform(finished_df["result"])


In [56]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# --- Classifier for match result ---
model_cls = RandomForestClassifier(n_estimators=300, random_state=42)
model_cls.fit(finished_df[features], y_cls)

# --- Regressors for home and away goals ---
home_goal_model = RandomForestRegressor(n_estimators=300, random_state=42)
away_goal_model = RandomForestRegressor(n_estimators=300, random_state=42)

home_goal_model.fit(finished_df[features], finished_df["team_h_score"])
away_goal_model.fit(finished_df[features], finished_df["team_a_score"])

print("✅ Models trained successfully.")


✅ Models trained successfully.


In [57]:
# === Predict top players likely to explode in next 3 gameweeks ===
players_df = pd.DataFrame(bootstrap_data["elements"])

# Keep only players available for selection
players_df = players_df[players_df["can_select"] & ~players_df["removed"]]

# Convert expected points and form to numeric
players_df["ep_next"] = pd.to_numeric(players_df["ep_next"], errors="coerce").fillna(0)
players_df["form"] = pd.to_numeric(players_df["form"], errors="coerce").fillna(0)

# Simple "explosion score" metric
players_df["explosion_score"] = players_df["ep_next"] * 0.6 + players_df["form"] * 0.4

# Sort and get top 15
top_players = players_df.sort_values("explosion_score", ascending=False).head(15)

# Map team names
top_players["team_name"] = top_players["team"].map(team_id_to_name)

print("🔥 Top 15 players likely to explode in the next gameweeks:\n")
display(top_players[[
    "web_name", "team_name", "element_type", 
    "form", "ep_next", "explosion_score"
]])


🔥 Top 15 players likely to explode in the next gameweeks:



,web_name,team_name,element_type,form,ep_next,explosion_score
473,Haaland,Man City,4,11.5,12.0,11.8
134,Semenyo,Bournemouth,3,10.8,10.8,10.8
4,Gabriel,Arsenal,2,8.2,8.7,8.5
431,Gravenberch,Liverpool,3,7.0,8.0,7.6
259,Caicedo,Chelsea,3,7.0,7.5,7.3
630,Alderete,Sunderland,2,7.5,7.0,7.2
525,Burn,Newcastle,2,6.5,7.0,6.8
463,Doku,Man City,3,6.0,6.5,6.3
448,Gvardiol,Man City,2,6.0,6.5,6.3
552,Woltemade,Newcastle,4,6.0,6.5,6.3


In [58]:

# --- Load entry history ---
with open("data/entry_history.json", "r", encoding="utf-8") as f:
    entry_history = json.load(f)

# --- Select the latest gameweek ---
latest_event = entry_history["current"][-1]

# --- Extract budget, transfers available, and hit points ---
budget = latest_event["bank"]  # budget in 0.1M units
transfers_available = latest_event["event_transfers"]  # free transfers available
hit_points = latest_event["event_transfers_cost"]      # points hit for extra transfers

print(f"💰 Budget: {budget/10:.1f}M")
print(f"🔄 Free transfers available: {transfers_available}")
print(f"⚠️ Points hit if extra transfers: {hit_points}")


💰 Budget: 2.6M
🔄 Free transfers available: 0
⚠️ Points hit if extra transfers: 0


In [59]:
import json
import pandas as pd

# --- CONFIG ---
PAST_GW = 7           # last completed gameweek
NEXT_GW = 8           # upcoming gameweek for transfer suggestions
LOOKAHEAD_WEEKS = 3   # how many GWs to look ahead
N_SUGGESTIONS = 3     # number of buy options per position

# --- Load player metadata ---
with open("data/bootstrap-static.json", "r", encoding="utf-8") as f:
    bootstrap_data = json.load(f)

players_df = pd.DataFrame(bootstrap_data["elements"])
players_df["price"] = players_df["now_cost"] / 10
players_df["expected_points"] = players_df["ep_next"].astype(float)
players_df["form"] = players_df["form"].astype(float)
players_df["position"] = players_df["element_type"].map({1: "GKP", 2: "DEF", 3: "MID", 4: "FWD"})
players_df["injured"] = players_df["status"].isin(["i", "d", "s"])
players_df["team_name"] = players_df["team"].map({t["id"]: t["name"] for t in bootstrap_data["teams"]})
players_df["projected_next3"] = players_df["expected_points"] * LOOKAHEAD_WEEKS

# --- Load your current team before transfers (PAST_GW) ---
with open(f"data/gw_picks/GW{PAST_GW}.json", "r", encoding="utf-8") as f:
    gw_team_data = json.load(f)

team_ids = [p["element"] for p in gw_team_data["picks"]]
team_df = players_df[players_df["id"].isin(team_ids)].copy()

# --- Money in the bank ---
money_in_bank = gw_team_data.get("entry_history", {}).get("bank", 0) / 10.0  # in millions

# --- Start suggestions ---
positions = ["GKP", "DEF", "MID", "FWD"]

print("═══════════════════════════════════════════════════════════════════")
print(f"📊 TRANSFER SUGGESTIONS for Gameweek {NEXT_GW} (Next {LOOKAHEAD_WEEKS} GWs)")
print("═══════════════════════════════════════════════════════════════════\n")
print(f"💰 Money in bank: £{money_in_bank:.1f}m\n")

for pos in positions:
    team_pos = team_df[team_df["position"] == pos]
    if team_pos.empty:
        continue

    weakest = team_pos.sort_values(["expected_points", "form"], ascending=[True, True]).iloc[0]
    sell_price = weakest["price"]
    total_budget = sell_price + money_in_bank

    # Filter viable replacement candidates
    candidates = players_df[
        (players_df["position"] == pos)
        & (~players_df["injured"])
        & (~players_df["id"].isin(team_ids))
        & (players_df["price"] <= total_budget)
    ].copy()

    candidates["score"] = candidates["expected_points"] * 0.6 + candidates["form"] * 0.4
    top_candidates = candidates.sort_values("score", ascending=False).head(N_SUGGESTIONS)

    weak_proj = weakest["projected_next3"]

    print(f"🧩 Position: {pos}")
    print(f"   ❌ SELL: {weakest.web_name} ({weakest.team_name})")
    print(f"      💰 Price: £{sell_price:.1f}m | 🔮 Expected (Next {LOOKAHEAD_WEEKS} GWs): {weak_proj:.1f} pts\n")

    print("   ✅ REPLACEMENT OPTIONS (within £{:.1f}m budget):".format(total_budget))
    for _, top in top_candidates.iterrows():
        top_proj = top["projected_next3"]
        proj_diff = top_proj - weak_proj

        print(f"      ➕ {top.web_name} ({top.team_name}) | £{top.price:.1f}m")
        print(f"         🔮 Exp: {top_proj:.1f} pts  (Δ {proj_diff:+.1f})")
        print("      ────────────────────────────────")
    print()

print("✅ Suggestions complete.")
print("═══════════════════════════════════════════════════════════════════")


═══════════════════════════════════════════════════════════════════
📊 TRANSFER SUGGESTIONS for Gameweek 8 (Next 3 GWs)
═══════════════════════════════════════════════════════════════════

💰 Money in bank: £2.6m

🧩 Position: GKP
   ❌ SELL: Sánchez (Chelsea)
      💰 Price: £4.9m | 🔮 Expected (Next 3 GWs): 3.0 pts

   ✅ REPLACEMENT OPTIONS (within £7.5m budget):
      ➕ Pope (Newcastle) | £5.1m
         🔮 Exp: 18.9 pts  (Δ +15.9)
      ────────────────────────────────
      ➕ Roefs (Sunderland) | £4.6m
         🔮 Exp: 16.5 pts  (Δ +13.5)
      ────────────────────────────────
      ➕ Raya (Arsenal) | £5.7m
         🔮 Exp: 13.5 pts  (Δ +10.5)
      ────────────────────────────────

🧩 Position: DEF
   ❌ SELL: De Cuyper (Brighton)
      💰 Price: £4.4m | 🔮 Expected (Next 3 GWs): 6.0 pts

   ✅ REPLACEMENT OPTIONS (within £7.0m budget):
      ➕ Gabriel (Arsenal) | £6.2m
         🔮 Exp: 26.1 pts  (Δ +20.1)
      ────────────────────────────────
      ➕ Alderete (Sunderland) | £4.1m
         🔮 Ex

In [60]:
import json
import pandas as pd
import itertools

# --- CONFIGURATION ---
PAST_GW = 3
NEXT_GW = 4
MAX_TRANSFERS = 3
HIT_COST = 4
LOOKAHEAD_WEEKS = 3
N_SUGGESTIONS = 5

# --- Load bootstrap player data ---
with open("data/bootstrap-static.json", "r", encoding="utf-8") as f:
    bootstrap_data = json.load(f)

players_df = pd.DataFrame(bootstrap_data["elements"])
players_df["price"] = players_df["now_cost"] / 10
players_df["expected_points"] = players_df["ep_next"].astype(float)
players_df["form"] = players_df["form"].astype(float)
players_df["position"] = players_df["element_type"].map({1: "GKP", 2: "DEF", 3: "MID", 4: "FWD"})
players_df["injured"] = players_df["status"].isin(["i", "d", "s"])

# --- Load your current team (before PAST_GW) ---
with open(f"data/gw_picks/GW{PAST_GW}.json", "r", encoding="utf-8") as f:
    gw_team_data = json.load(f)

team_ids = [p["element"] for p in gw_team_data["picks"]]
team_df = players_df[players_df["id"].isin(team_ids)].copy()

# --- Identify weakest players ---
team_df["weak_index"] = (10 - team_df["expected_points"]) * 0.6 + (10 - team_df["form"]) * 0.4
weakest_players = team_df.sort_values("weak_index", ascending=False).head(MAX_TRANSFERS)

# --- Load actual next GWs performance (GW4 → GW6) ---
actual_points = {}
for gw in range(NEXT_GW, NEXT_GW + LOOKAHEAD_WEEKS):
    try:
        with open(f"data/element_gameweek_live/GW{gw}.json", "r", encoding="utf-8") as f:
            gw_data = json.load(f)
        df = pd.DataFrame(gw_data["elements"])
        df["id"] = df.index + 1
        for _, row in df.iterrows():
            stats = row["stats"]
            actual_points[row["id"]] = actual_points.get(row["id"], 0) + stats["total_points"]
    except FileNotFoundError:
        pass  # skip missing GWs

# --- Helper to get actual points (3 GWs) ---
def get_actual_points(player_id):
    return actual_points.get(player_id, 0)

# --- Suggest replacements for each weak player ---
suggestions = []
for _, weak in weakest_players.iterrows():
    replacements = players_df[
        (players_df["position"] == weak["position"]) &
        (~players_df["injured"]) &
        (~players_df["id"].isin(team_ids))
    ].copy()
    replacements["score"] = replacements["expected_points"] * 0.6 + replacements["form"] * 0.4
    top_repls = replacements.sort_values("score", ascending=False).head(N_SUGGESTIONS)
    for _, repl in top_repls.iterrows():
        proj_diff = (repl["expected_points"] * LOOKAHEAD_WEEKS) - (weak["expected_points"] * LOOKAHEAD_WEEKS)
        actual_diff = get_actual_points(repl["id"]) - get_actual_points(weak["id"])
        suggestions.append({
            "out": weak["web_name"],
            "in": repl["web_name"],
            "proj_diff": proj_diff,
            "actual_diff": actual_diff
        })

# --- Build valid combinations (no duplicate sold players) ---
options = []
option_num = 1

for n in range(1, MAX_TRANSFERS + 1):
    for combo in itertools.combinations(suggestions, n):
        # ✅ skip combos selling the same player more than once
        sold_players = [c["out"] for c in combo]
        if len(sold_players) != len(set(sold_players)):
            continue

        total_proj_gain = sum(c["proj_diff"] for c in combo)
        total_actual_gain = sum(c["actual_diff"] for c in combo)
        hit_cost = (n - 1) * HIT_COST if n > 1 else 0
        options.append({
            "option_num": option_num,
            "transfers": combo,
            "n_transfers": n,
            "proj_gain": total_proj_gain,
            "actual_gain": total_actual_gain,
            "after_hit_proj": total_proj_gain - hit_cost,
            "after_hit_actual": total_actual_gain - hit_cost
        })
        option_num += 1

# --- Sort and display ---
options = sorted(options, key=lambda x: x["after_hit_proj"], reverse=True)

print(f"📈 Suggested Transfer Strategies (with actual results from GW{NEXT_GW}-{NEXT_GW + LOOKAHEAD_WEEKS - 1}):\n")

for opt in options[:10]:
    print(f"⚡ Option {opt['option_num']}: {opt['n_transfers']} transfers (-{(opt['n_transfers']-1)*HIT_COST})")
    for t in opt["transfers"]:
        print(f"   🔄 {t['out']} ➜ {t['in']} | +{t['proj_diff']:.1f} pts projected | Actual diff: +{t['actual_diff']:.1f}")
    print(f"   💹 Total Projected Gain: +{opt['proj_gain']:.1f} | Actual Gain: +{opt['actual_gain']:.1f}")
    print(f"   🧾 After Hit: Projected +{opt['after_hit_proj']:.1f} | Actual +{opt['after_hit_actual']:.1f}")
    print("—" * 80)


📈 Suggested Transfer Strategies (with actual results from GW4-6):

⚡ Option 91: 3 transfers (-8)
   🔄 Sánchez ➜ Pope | +15.9 pts projected | Actual diff: +16.0
   🔄 Wood ➜ Haaland | +32.1 pts projected | Actual diff: +33.0
   🔄 De Cuyper ➜ Gabriel | +20.1 pts projected | Actual diff: +15.0
   💹 Total Projected Gain: +68.1 | Actual Gain: +64.0
   🧾 After Hit: Projected +60.1 | Actual +56.0
————————————————————————————————————————————————————————————————————————————————
⚡ Option 116: 3 transfers (-8)
   🔄 Sánchez ➜ Roefs | +13.5 pts projected | Actual diff: +22.0
   🔄 Wood ➜ Haaland | +32.1 pts projected | Actual diff: +33.0
   🔄 De Cuyper ➜ Gabriel | +20.1 pts projected | Actual diff: +15.0
   💹 Total Projected Gain: +65.7 | Actual Gain: +70.0
   🧾 After Hit: Projected +57.7 | Actual +62.0
————————————————————————————————————————————————————————————————————————————————
⚡ Option 92: 3 transfers (-8)
   🔄 Sánchez ➜ Pope | +15.9 pts projected | Actual diff: +16.0
   🔄 Wood ➜ Haaland | +32.

In [61]:
import json
import pandas as pd
import itertools
import os
from collections import defaultdict

# --- CONFIGURATION ---
CURRENT_GW = 7            # GW before which you make transfers
LOOKAHEAD_WEEKS = 3       # how many GWs ahead to predict (e.g. GW7..GW9)
MAX_TRANSFERS = 3
HIT_COST = 4              # -4 points per extra transfer beyond first
N_SUGGESTIONS = 6         # top replacements to consider per weak player
PAST_FORM_MATCHES = 3     # how many past matches to use to compute team form

# --- Load bootstrap-static players & teams ---
with open("data/bootstrap-static.json", "r", encoding="utf-8") as f:
    bootstrap = json.load(f)

players_df = pd.DataFrame(bootstrap["elements"])
teams_df = pd.DataFrame(bootstrap["teams"])[["id", "name"]]

# standard columns / helper columns
players_df["price"] = players_df["now_cost"] / 10
players_df["expected_points"] = players_df["ep_next"].astype(float)
players_df["form"] = players_df["form"].astype(float)
players_df["position"] = players_df["element_type"].map({1: "GKP", 2: "DEF", 3: "MID", 4: "FWD"})
players_df["injured"] = players_df["status"].isin(["i", "d", "s"])  # injured/doubt/suspended
players_df["team_name"] = players_df["team"].map(teams_df.set_index("id")["name"])

# --- Build fixtures_df by reading all GW files in data/gw_data/ ---
gw_folder = "data/gw_data"
all_matches = []
for fn in sorted(os.listdir(gw_folder)):
    if fn.startswith("GW") and fn.endswith(".json"):
        gw_num = int(fn.replace("GW", "").replace(".json", ""))
        with open(os.path.join(gw_folder, fn), "r", encoding="utf-8") as f:
            match_list = json.load(f)
        # match_list may be a list of fixtures (as your sample)
        for m in match_list:
            # attach event if present or use gw_num
            m_event = m.get("event", gw_num)
            m["event"] = m_event
            all_matches.append(m)

fixtures_df = pd.DataFrame(all_matches)

# normalize missing difficulty fields
fixtures_df["team_h_difficulty"] = fixtures_df.get("team_h_difficulty").fillna(3)
fixtures_df["team_a_difficulty"] = fixtures_df.get("team_a_difficulty").fillna(3)

# --- Compute team form based on past matches (last PAST_FORM_MATCHES matches before CURRENT_GW) ---
# We'll compute simple points (3 win,1 draw) per match for each team then normalize.
def team_points_from_match(row, team_id):
    # returns points that team_id gained in this match row
    if not row.get("finished", False):
        return 0
    th, ta = row.get("team_h"), row.get("team_a")
    sh, sa = row.get("team_h_score"), row.get("team_a_score")
    if team_id == th:
        if sh is None or sa is None:
            return 0
        if sh > sa: return 3
        if sh == sa: return 1
        return 0
    if team_id == ta:
        if sh is None or sa is None:
            return 0
        if sa > sh: return 3
        if sa == sh: return 1
        return 0
    return 0

team_form_scores = {}
team_ids = teams_df["id"].tolist()
for tid in team_ids:
    # filter matches for this team with event < CURRENT_GW
    team_matches = fixtures_df[
        ((fixtures_df["team_h"] == tid) | (fixtures_df["team_a"] == tid)) &
        (fixtures_df["event"] < CURRENT_GW) &
        (fixtures_df.get("finished", False) == True)
    ].sort_values("event", ascending=False).head(PAST_FORM_MATCHES)
    pts = sum(team_points_from_match(r, tid) for _, r in team_matches.iterrows())
    # normalize to [0,1] where max possible is 3*PAST_FORM_MATCHES
    norm = pts / (3 * max(1, len(team_matches)))
    team_form_scores[tid] = norm

teams_df["form_score"] = teams_df["id"].map(team_form_scores).fillna(0)

# merge team form into players_df
players_df = players_df.merge(teams_df[["id", "form_score"]], left_on="team", right_on="id", how="left", suffixes=("", "_team"))
players_df.drop(columns=["id_team"], inplace=True)

# --- Build future_fixtures for CURRENT_GW .. CURRENT_GW + LOOKAHEAD_WEEKS -1 ---
future_events = list(range(CURRENT_GW, CURRENT_GW + LOOKAHEAD_WEEKS))
future_fixtures = fixtures_df[fixtures_df["event"].isin(future_events)].copy()

# helper: compute average fixture difficulty (higher=harder) for a team across the next LOOKAHEAD_WEEKS
def avg_fixture_difficulty_for_team(team_id):
    team_fixtures = future_fixtures[
        (future_fixtures["team_h"] == team_id) | (future_fixtures["team_a"] == team_id)
    ].head(LOOKAHEAD_WEEKS)
    if team_fixtures.empty:
        return 3.0  # neutral
    diffs = []
    for _, r in team_fixtures.iterrows():
        if r["team_h"] == team_id:
            diffs.append(r.get("team_h_difficulty", 3))
        else:
            diffs.append(r.get("team_a_difficulty", 3))
    return float(sum(diffs) / max(1, len(diffs)))

players_df["avg_fdr"] = players_df["team"].apply(avg_fixture_difficulty_for_team)

# --- Rotation risk estimate (simple) ---
# use minutes and starts if present; fallback to no risk
players_df["minutes"] = players_df.get("minutes", 0).fillna(0).astype(float)
players_df["starts"] = players_df.get("starts", 0).fillna(0).astype(float)
# minutes per start; if starts==0, use minutes (low confidence)
players_df["minutes_per_start"] = players_df.apply(lambda r: (r["minutes"] / r["starts"]) if r["starts"] > 0 else r["minutes"], axis=1)
players_df["rotation_risk"] = (players_df["minutes_per_start"] < 60).astype(int)  # 1 = risk
players_df["rotation_penalty"] = players_df["rotation_risk"] * 1.5  # penalty applied to projected points

# --- Load current team picks for CURRENT_GW (team before transfers) ---
with open(f"data/gw_picks/GW{CURRENT_GW}.json", "r", encoding="utf-8") as f:
    gw_picks = json.load(f)
current_team_ids = [p["element"] for p in gw_picks["picks"]]

# --- bank/free transfers/hit from entry_history (if exists) ---
try:
    with open("data/entry_history.json", "r", encoding="utf-8") as f:
        entry_history = json.load(f)
    entry = entry_history["current"][-1]
    bank = entry["bank"] / 10
    free_transfers = entry["event_transfers"]
    current_hit = entry["event_transfers_cost"]
except Exception:
    bank = 0.0
    free_transfers = 0
    current_hit = 0

# --- prediction function for LOOKAHEAD_WEEKS ---
def predict_points_for_lookahead(row):
    # base = expected_points per next match (ep_next)
    base_ep = float(row["expected_points"])
    # form factor (0.8..1.3)
    form_factor = 1.0 + (row["form"] / 10.0) * 0.15
    # team form factor (0..0.2)
    team_factor = 1.0 + (row.get("form_score", 0.0) * 0.2)
    # fixture difficulty factor: easier fixtures -> multiply up, scaled so avg diff 3 -> 1.0
    # We convert avg_fdr to an ease score: ease = (6 - avg_fdr)/5 in range (0.2..1)
    ease = (6.0 - float(row.get("avg_fdr", 3.0))) / 5.0
    ease_factor = 0.8 + 0.4 * ease   # between 0.8 and 1.2
    # rotation penalty
    rotation_pen = float(row.get("rotation_penalty", 0.0))
    # projected points over LOOKAHEAD_WEEKS
    projected = base_ep * LOOKAHEAD_WEEKS * form_factor * team_factor * ease_factor
    projected = projected - rotation_pen * LOOKAHEAD_WEEKS
    return float(max(0.0, round(projected, 2)))

players_df["predicted_points_3gw"] = players_df.apply(predict_points_for_lookahead, axis=1)

# --- current team subset and weak players selection ---
team_df = players_df[players_df["id"].isin(current_team_ids)].copy()
team_df["weak_index"] = (10 - team_df["expected_points"]) * 0.5 + (10 - team_df["form"]) * 0.3 + (1 - team_df["form_score"]) * 0.2
weak_candidates = team_df.sort_values("weak_index", ascending=False).head(max(3, MAX_TRANSFERS*2))

# --- Available candidates for buying ---
available_players = players_df[
    (~players_df["injured"]) &
    (players_df["form"] > 0) &
    (~players_df["id"].isin(current_team_ids))
].copy()

# --- For each weak player, compute top replacements (by predicted_points_3gw) ---
replacement_suggestions = []  # list of dicts: {out_id,out_name,out_pos, in_id,in_name,in_pos,proj_diff}
for _, weak in weak_candidates.iterrows():
    candidates = available_players[available_players["position"] == weak["position"]].copy()
    # require that candidate price is not insane: allow candidate.price <= weak.price + bank (independent suggestions)
    candidates = candidates[candidates["price"] <= (weak["price"] + bank)]
    # rank by predicted_points_3gw
    candidates = candidates.sort_values("predicted_points_3gw", ascending=False).head(N_SUGGESTIONS)
    for _, buy in candidates.iterrows():
        proj_diff = buy["predicted_points_3gw"] - weak["predicted_points_3gw"] if "predicted_points_3gw" in weak else buy["predicted_points_3gw"] - (weak["expected_points"]*LOOKAHEAD_WEEKS)
        replacement_suggestions.append({
            "out_id": int(weak["id"]),
            "out_name": weak["web_name"],
            "out_pos": weak["position"],
            "out_price": weak["price"],
            "in_id": int(buy["id"]),
            "in_name": buy["web_name"],
            "in_pos": buy["position"],
            "in_price": buy["price"],
            "proj_gain": round(proj_diff, 2)
        })

# --- Generate transfer bundles (1..MAX_TRANSFERS) ensuring:
#    - we don't sell the same player twice in a bundle
#    - we don't buy the same player twice in a bundle
#    - respect per-bundle affordability using independent budget (bank + sale price per swap)
options = []
option_counter = 1

# pre-build suggestions grouped by out_id to avoid selling a player multiple times in combination generation
# but we still iterate combos across suggestion list and then filter duplicates
for n in range(1, MAX_TRANSFERS + 1):
    for combo in itertools.combinations(replacement_suggestions, n):
        sold_ids = [c["out_id"] for c in combo]
        bought_ids = [c["in_id"] for c in combo]
        # skip if same player sold twice or same player bought twice
        if len(set(sold_ids)) != len(sold_ids): 
            continue
        if len(set(bought_ids)) != len(bought_ids):
            continue
        # check affordability independently for each pair (we treat each pair as independent: bank + out_price must cover in_price)
        affordable = True
        for c in combo:
            available_money = bank + c["out_price"]
            if c["in_price"] > available_money + 1e-9:  # small tolerance
                affordable = False
                break
        if not affordable:
            continue
        total_proj = sum(c["proj_gain"] for c in combo)
        hit_penalty = HIT_COST * max(0, (len(combo) - free_transfers))
        net_proj = total_proj - hit_penalty
        options.append({
            "option_num": option_counter,
            "n_transfers": len(combo),
            "hit": hit_penalty,
            "proj_gain": round(total_proj, 2),
            "net_proj": round(net_proj, 2),
            "details": combo
        })
        option_counter += 1

# sort options by net_proj descending
options = sorted(options, key=lambda x: x["net_proj"], reverse=True)

# --- Print top options ---
print(f"📅 Predicting GW{CURRENT_GW} → GW{CURRENT_GW + LOOKAHEAD_WEEKS - 1}")
print(f"💰 Bank: {bank}M | 🔄 Free transfers: {free_transfers} | ⚡ Current hit: {current_hit}\n")
print("📈 Suggested transfer bundles (projected points over next {} GWs):\n".format(LOOKAHEAD_WEEKS))

if not options:
    print("No feasible transfer bundles found under current constraints.")
else:
    shown = 0
    for opt in options:
        shown += 1
        print(f"⚡ Option {opt['option_num']}: {opt['n_transfers']} transfer{'s' if opt['n_transfers']>1 else ''} "
              f"({'-{}'.format(opt['hit']) if opt['hit'] else 'No hit'})")
        for c in opt["details"]:
            print(f"   🔄 {c['out_name']} ➜ {c['in_name']} | +{c['proj_gain']:.2f} pts projected")
        print(f"   💹 Total Projected Gain (raw): +{opt['proj_gain']:.2f} pts")
        print(f"   🧾 After Hit: +{opt['net_proj']:.2f} pts\n")
        print("-" * 80)
        if shown >= 10:
            break


📅 Predicting GW7 → GW9
💰 Bank: 2.6M | 🔄 Free transfers: 0 | ⚡ Current hit: 0

📈 Suggested transfer bundles (projected points over next 3 GWs):

⚡ Option 561: 3 transfers (-12)
   🔄 Fraser ➜ Calvert-Lewin | +15.95 pts projected
   🔄 Sánchez ➜ Pope | +20.06 pts projected
   🔄 De Cuyper ➜ Gabriel | +29.08 pts projected
   💹 Total Projected Gain (raw): +65.09 pts
   🧾 After Hit: +53.09 pts

--------------------------------------------------------------------------------
⚡ Option 863: 3 transfers (-12)
   🔄 Fraser ➜ Thiago | +15.90 pts projected
   🔄 Sánchez ➜ Pope | +20.06 pts projected
   🔄 De Cuyper ➜ Gabriel | +29.08 pts projected
   💹 Total Projected Gain (raw): +65.04 pts
   🧾 After Hit: +53.04 pts

--------------------------------------------------------------------------------
⚡ Option 582: 3 transfers (-12)
   🔄 Fraser ➜ Calvert-Lewin | +15.95 pts projected
   🔄 Sánchez ➜ Roefs | +17.57 pts projected
   🔄 De Cuyper ➜ Gabriel | +29.08 pts projected
   💹 Total Projected Gain (raw): +

In [68]:
import json
import pandas as pd
import os

# ------------------ CONFIG ------------------
LOOKAHEAD_GWS = 3
PLAYER_NAME = "João Pedro"  # Example player

# ------------------ LOAD PLAYERS ------------------
with open("data/bootstrap-static.json", "r", encoding="utf-8") as f:
    bootstrap_data = json.load(f)

players_df = pd.DataFrame(bootstrap_data["elements"])
players_df["price"] = players_df["now_cost"] / 10
players_df["expected_points"] = players_df["ep_next"].astype(float)
players_df["form"] = players_df["form"].astype(float)
players_df["position"] = players_df["element_type"].map({1: "GKP", 2: "DEF", 3: "MID", 4: "FWD"})
players_df["injured"] = players_df["status"].isin(["i", "d", "s"])

# ------------------ HELPER FUNCTIONS ------------------
def search_player_by_name(name, df):
    results = df[df["web_name"].str.contains(name, case=False)]
    if results.empty:
        print(f"No player found with name '{name}'")
        return None
    print("Found player(s):")
    print(results[["id", "web_name", "team", "position", "expected_points", "form"]])
    return results.iloc[0]

def load_all_gw_data():
    """Load all element_gameweek_live JSON files into a dict of DataFrames"""
    gw_files = sorted([f for f in os.listdir("data/element_gameweek_live") if f.endswith(".json")])
    all_gws = {}
    for gw_file in gw_files:
        gw_num = int(gw_file.replace("GW","").replace(".json",""))
        with open(f"data/element_gameweek_live/{gw_file}", "r", encoding="utf-8") as f:
            gw_data = json.load(f)
        df = pd.DataFrame(gw_data["elements"])
        df["id"] = df["id"].astype(int)
        all_gws[gw_num] = df
    return all_gws

# ------------------ PREDICT PLAYER POINTS ------------------
def predict_player_points(player_name, players_df, lookahead_gws=3):
    player = search_player_by_name(player_name, players_df)
    if player is None:
        return
    
    base_ep = player["expected_points"]
    form = player["form"]
    position = player["position"]
    team = player["team"]
    player_id = player["id"]
    
    # Load all GW data
    all_gws = load_all_gw_data()
    
    proj_list = []
    for gw_num, df in all_gws.items():
        row = df[df["id"] == player_id]
        if row.empty:
            continue
        row = row.iloc[0]
        stats = row["stats"]
        goals = stats.get("goals_scored", 0)
        assists = stats.get("assists", 0)
        xG = float(stats.get("expected_goals", 0))
        xA = float(stats.get("expected_assists", 0))
        xGC = float(stats.get("expected_goals_conceded", 0))
        saves = stats.get("saves", 0)
        def_contrib = stats.get("defensive_contribution", 0)
        big_chances = stats.get("big_chances_created", 0)
        total_points = stats.get("total_points", 0)
        
        # ------------------ REALISTIC PROJECTED POINTS ------------------
        if position == "GKP":
            projected = base_ep + (form/10) + (saves*1) - (xGC*2)
        elif position == "DEF":
            projected = base_ep + (form/10) + (def_contrib*0.5) - (xGC*1.5)
        elif position in ["MID", "FWD"]:
            projected = base_ep + (form/10) + (xG*4) + (xA*3) + (big_chances*0.5)
        else:
            projected = base_ep

        proj_list.append({
            "GW": gw_num,
            "projected_points": round(projected,1),
            "total_points": total_points,
            "goals": goals,
            "assists": assists,
            "xG": xG,
            "xA": xA,
            "xGC": xGC,
            "saves": saves,
            "def_contrib": def_contrib,
            "big_chances": big_chances
        })
    
    proj_df = pd.DataFrame(proj_list).sort_values("GW")
    
    print(f"\nPlayer: {player['web_name']} | Pos: {position} | Team: {team} | Form: {form} | Base EP: {base_ep}")
    print("\nProjected points for all played GWs this season:\n")
    print(proj_df[[
        "GW","projected_points","total_points","goals","assists","xG","xA","xGC","saves","def_contrib","big_chances"
    ]])
    
    # Next N GW projection
    next3_proj = proj_df.tail(lookahead_gws)["projected_points"].sum()
    print(f"\nProjected points next {lookahead_gws} GWs: {next3_proj:.1f}")
    print(f"Total actual points so far: {proj_df['total_points'].sum()}")

# ------------------ RUN ------------------
predict_player_points(PLAYER_NAME, players_df, LOOKAHEAD_GWS)


Found player(s):
      id    web_name  team position  expected_points  form
266  249  João Pedro     7      FWD              3.7   3.2

Player: João Pedro | Pos: FWD | Team: 7 | Form: 3.2 | Base EP: 3.7

Projected points for all played GWs this season:

   GW  projected_points  total_points  goals  assists    xG    xA   xGC  \
0   1               5.2             2      0        0  0.28  0.02  0.60   
1   2               6.0            15      1        2  0.41  0.11  0.73   
2   3               6.7             9      1        0  0.66  0.02  1.02   
3   4               4.7             7      0        1  0.15  0.02  0.71   
4   5               4.1             2      0        0  0.00  0.03  1.84   
5   6               4.1             2      0        0  0.00  0.03  2.28   
6   7               4.1             2      0        0  0.03  0.00  1.54   

   saves  def_contrib  big_chances  
0      0            1            0  
1      0            5            0  
2      0           10            0